In [1]:
#Imports
import pandas as pd
import os
import tensorflow as tf
from keras import backend as K

from utils.modelgenerator import *
from utils.modelhandler import *
from utils.datahandler import *

#Get data 
cwd = os.path.normpath(os.getcwd())
df = pd.read_csv(cwd+'/data/df_with_final_features.csv', index_col='Date') #df = pd.read_csv('user5.csv')
df.index = pd.to_datetime(df.index)
#df = df[['User5', 'temp', 'rhum']]
df.fillna(0, inplace=True)

df_array = []
for idx in range(30):
    df_array.append(df[[f'User{idx+1}', 'temp', 'rhum', 'wspd', 'PC1', 'hour sin', 'hour cos', f'User{idx+1}_lag_24hrs']])

#df_array[3].head(3)

#Train, Validation and Test datasets
sequence_length = 25
batch_size = 16
num_features = df_array[0].shape[1]

dh = Datahandler()

X_train, y_train, X_val, y_val, X_test, y_test = {}, {}, {}, {}, {}, {}

#Create Train, Validation and Test datasets
for idx, df in enumerate(df_array):
    n = len(df)
    train_df = df[0:int(n*0.7)]
    val_df = df[int(n*0.7):int(n*0.9)]
    test_df = df[int(n*0.9):]

    # Min max sclaing
    train_df = dh.min_max_scaling(train_df)
    val_df = dh.min_max_scaling(val_df)
    test_df = dh.min_max_scaling(test_df)

    # Sequencing
    train_sequences = dh.create_sequences(train_df, sequence_length)
    val_sequences = dh.create_sequences(val_df, sequence_length)
    test_sequences = dh.create_sequences(test_df, sequence_length)

    #Split into feature and label
    X_train[f'user{idx+1}'], y_train[f'user{idx+1}'] = dh.prepare_data(train_sequences, batch_size)
    X_val[f'user{idx+1}'], y_val[f'user{idx+1}'] = dh.prepare_data(val_sequences, batch_size)
    X_test[f'user{idx+1}'], y_test[f'user{idx+1}'] = dh.prepare_data(test_sequences, batch_size)

#General Hyperparameters
# #All models
horizon = 1
max_epochs = 100
m1 = ModelGenerator()
mh = Modelhandler()

loss = tf.keras.losses.MeanSquaredError()
metrics=[
    tf.keras.metrics.RootMeanSquaredError(), 
    tf.keras.metrics.MeanAbsolutePercentageError(),
    tf.keras.metrics.MeanAbsoluteError(),
]

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=10,mode='min')
timing_callback = TimingCallback()
custom_callback = CustomCallback()
#model_checkpoint = ModelCheckpoint('models/best_model.h5', save_best_only=True, monitor='val_loss', mode='min')
callbacks=[early_stopping, timing_callback, custom_callback] #model_checkpoint


In [2]:
def sum_weights(weight_list):
    """
    Return the sum of the listed weights. The is equivalent to avg of the weights
    """
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*weight_list):
        layer_mean = tf.math.reduce_mean(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)
        
    return avg_grad

# Federated Learning Dense MoE Model

In [3]:
dense_results = pd.DataFrame(columns=['architecture', 'train_time', 'avg_time_epoch', 'mse','mse_std', 'rmse','rmse_std','mape','mape_std','mae','mae_std'])

In [4]:
#MOE MODEL 1 ------------------------------------------------------------------
dense_moe_architecture = "Dense_MoE_e4d8_d16"
dense_units = 16
num_experts = 4
expert_units = 8

#Build and save global model
global_dense_moe = m1.build_soft_dense_moe_model(X_train[f'user{1}'], batch_size, horizon, dense_units, expert_units, num_experts, m1)
global_dense_moe.save(cwd + f"/models/FL/Dense/global_dense_model/{dense_moe_architecture}/FederatedRound{0}")

federated_rounds = 5
for federated_round  in range(federated_rounds):
    print("Started Federated training round ----------", federated_round+1, f"/ {federated_rounds}")

    #Get global models weights
    global_dense_moe = keras.models.load_model(cwd + f"/models/FL/Dense/global_dense_model/{dense_moe_architecture}/FederatedRound{federated_round}", compile=False)
    global_dense_moe_weights = global_dense_moe.get_weights()

    #initial list for local model weights
    local_dense_moe_weight_list = list()

    for idx, user in enumerate(df_array): 
        print("User: ", idx+1)
        
        #build and compile local model X_train, batch_size, horizon, dense_units,  expert_units, num_experts, m1
        local_dense_moe_model = m1.build_soft_dense_moe_model(X_train[f'user{idx+1}'], batch_size, horizon, dense_units, expert_units, num_experts, m1)
        local_dense_moe_model.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=metrics)
        
        #set local model weight to the weight of the global model
        local_dense_moe_model.set_weights(global_dense_moe_weights)
        
        #Fit local model to local data
        dense_histroy, dense_user_results = mh.compile_fit_evaluate_model(
            model=local_dense_moe_model, 
            loss=loss, 
            metrics=metrics, 
            X_train=X_train[f'user{idx+1}'],
            y_train = y_train[f'user{idx+1}'], 
            max_epochs = max_epochs, 
            batch_size=batch_size, 
            X_val=X_val[f'user{idx+1}'], 
            y_val=y_val[f'user{idx+1}'], 
            X_test=X_test[f'user{idx+1}'], 
            y_test=y_test[f'user{idx+1}'], 
            callbacks=callbacks, 
            user=f'user{idx+1}', 
            hyper=dense_moe_architecture,
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        )
        #add model weights to list        
        local_dense_moe_weights = local_dense_moe_model.get_weights()
        local_dense_moe_weight_list.append(local_dense_moe_weights)
      
        #clear session to free memory after each communication round
        K.clear_session()
    
    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights_dense_moe = sum_weights(local_dense_moe_weight_list)
    #update global model 
    global_dense_moe.set_weights(average_weights_dense_moe)
    #Save global models
    global_dense_moe.save(cwd + f"/models/FL/Dense/global_dense_model/{dense_moe_architecture}/FederatedRound{federated_round+1}")
    print("Saved Global models")

dense_all_results = pd.DataFrame(columns=["user", "architecture", "train_time", "avg_time_epoch", "mse", "rmse", "mape", "mae"])

#For each of the users
for idx in range(len(df_array)):
    print("User: ", idx+1)
    for round in range(3):
        #print("Round: ", round)
        global_dense_moe = tf.keras.models.load_model(cwd + f"/models/FL/Dense/global_dense_model/{dense_moe_architecture}/FederatedRound{federated_rounds}", compile=False)
        dense_moe_model = m1.build_soft_dense_moe_model(X_train[f'user{idx+1}'], batch_size, horizon, dense_units, expert_units, num_experts, m1)
        dense_moe_model.set_weights(global_dense_moe.get_weights())
        
        dense_histroy, dense_user_results = mh.compile_fit_evaluate_model(
            model=dense_moe_model, 
            loss=loss, 
            metrics=metrics, 
            X_train=X_train[f'user{idx+1}'],
            y_train = y_train[f'user{idx+1}'], 
            max_epochs = max_epochs, 
            batch_size=batch_size, 
            X_val=X_val[f'user{idx+1}'], 
            y_val=y_val[f'user{idx+1}'], 
            X_test=X_test[f'user{idx+1}'], 
            y_test=y_test[f'user{idx+1}'], 
            callbacks=callbacks, 
            user=f'user{idx+1}', 
            hyper=dense_moe_architecture,
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
        )
        # Add the 'architecture' column from dense_user_results to dense_results
        dense_all_results = pd.merge(dense_all_results, dense_user_results, how='outer')   
        
for idx in range(len(df_array)):
    new_row = {
        'architecture': dense_moe_architecture,
        'train_time': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["train_time"].mean(), 
        'avg_time_epoch' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["avg_time_epoch"].mean(),
        'mse': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mse"].mean(),
        'mse_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mse"].std(),
        'rmse': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["rmse"].mean(),
        'rmse_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["rmse"].std(),
        'mape': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mape"].mean(),
        'mape_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mape"].std(),
        'mae': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mae"].mean(),
        'mae_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mae"].std(),
    }
    dense_results.loc[len(dense_results)] = new_row

INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/Dense_MoE_e4d8_d16/FederatedRound0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/Dense_MoE_e4d8_d16/FederatedRound0\assets


Started Federated training round ---------- 1 / 5
User:  1
User:  2
User:  3
User:  4
User:  5
User:  6
User:  7
User:  8
User:  9
User:  10
User:  11
User:  12
User:  13
User:  14
User:  15
User:  16
User:  17
User:  18
User:  19
User:  20
User:  21
User:  22
User:  23
User:  24
User:  25
User:  26
User:  27
User:  28
User:  29
User:  30


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/Dense_MoE_e4d8_d16/FederatedRound1\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/Dense_MoE_e4d8_d16/FederatedRound1\assets


...

INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/Dense_MoE_e4d8_d16/FederatedRound4\assets


Saved Global models
Started Federated training round ---------- 5 / 5
User:  1
User:  2
User:  3
User:  4
User:  5
User:  6
User:  7
User:  8
User:  9
User:  10
User:  11
User:  12
User:  13
User:  14
User:  15
User:  16
User:  17
User:  18
User:  19
User:  20
User:  21
User:  22
User:  23
User:  24
User:  25
User:  26
User:  27
User:  28
User:  29
User:  30


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/Dense_MoE_e4d8_d16/FederatedRound5\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/Dense_MoE_e4d8_d16/FederatedRound5\assets


Saved Global models
User:  1
User:  2
User:  3
User:  4
User:  5
User:  6
User:  7
User:  8
User:  9
User:  10
User:  11
User:  12
User:  13
User:  14
User:  15
User:  16
User:  17
User:  18
User:  19
User:  20
User:  21
User:  22
User:  23
User:  24
User:  25
User:  26
User:  27
User:  28
User:  29
User:  30


In [5]:
dense_results.to_csv(f'evaluations/federated_learning/{dense_moe_architecture}.csv')
dense_results

,architecture,train_time,avg_time_epoch,mse,mse_std,rmse,rmse_std,mape,mape_std,mae,mae_std
0,Dense_MoE_e4d8_d16,9.268465,0.511754,0.032702,0.000444,0.180834,0.001227,1.250649e+05,12389.254496,0.128126,0.001071
1,Dense_MoE_e4d8_d16,8.605041,0.532423,0.018393,0.000401,0.135615,0.001475,1.654385e+05,40602.250888,0.081686,0.004076
2,Dense_MoE_e4d8_d16,13.408503,0.509677,0.020054,0.000587,0.141602,0.002074,4.704948e+04,2050.326539,0.078684,0.003227
3,Dense_MoE_e4d8_d16,6.529357,0.536536,0.021754,0.000949,0.147468,0.003220,1.313221e+05,11123.619260,0.063685,0.001449
4,Dense_MoE_e4d8_d16,6.800499,0.545564,0.014152,0.000337,0.118955,0.001415,1.051527e+05,26613.256395,0.076650,0.003925
5,Dense_MoE_e4d8_d16,7.582706,0.523895,0.019952,0.001053,0.141219,0.003700,7.669271e+04,16555.423893,0.093478,0.000894
6,Dense_MoE_e4d8_d16,12.690824,0.497024,0.022740,0.000921,0.150778,0.003038,9.930006e+04,8539.451449,0.085347,0.000249
7,Dense_MoE_e4d8_d16,8.062499,0.524665,0.026485,0.000438,0.162740,0.001341,2.144867e+04,19135.500169,0.106464,0.001378
8,Dense_MoE_e4d8_d16,6.512782,0.535089,0.024701,0.000680,0.157155,0.002156,9.044453e+01,10.354164,0.096917,0.004208
9,Dense_MoE_e4d8_d16,8.307605,0.525713,0.016535,0.001275,0.128526,0.004909,1.289280e+06,270636.507982,0.069106,0.002358


In [6]:
#MOE MODEL 2 ------------------------------------------------------------------
dense_moe_architecture = "TopK_Dense_MoE" #top2_exp5_d8
dense_units = 16

num_experts = 5
expert_units = 8
top_k = 2

#Build and save global model
global_dense_moe = m1.build_topk_dense_moe_model(X_train[f'user{1}'], batch_size, horizon, dense_units, num_experts, top_k, expert_units, m1)
global_dense_moe.save(cwd + f"/models/FL/Dense/global_dense_model/{dense_moe_architecture}/FederatedRound{0}")

federated_rounds = 5
for federated_round  in range(federated_rounds):
    print("Started Federated training round ----------", federated_round+1, f"/ {federated_rounds}")

    #Get global models weights
    global_dense_moe = keras.models.load_model(cwd + f"/models/FL/Dense/global_dense_model/{dense_moe_architecture}/FederatedRound{federated_round}", compile=False)
    global_dense_moe_weights = global_dense_moe.get_weights()

    #initial list for local model weights
    local_dense_moe_weight_list = list()

    for idx, user in enumerate(df_array): 
        print("User: ", idx+1)
        
        #build and compile local model X_train, batch_size, horizon, dense_units,  expert_units, num_experts, m1
        local_dense_moe_model = m1.build_topk_dense_moe_model(X_train[f'user{idx+1}'], batch_size, horizon, dense_units, num_experts, top_k, expert_units, m1)
        local_dense_moe_model.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=metrics)
        
        #set local model weight to the weight of the global model
        local_dense_moe_model.set_weights(global_dense_moe_weights)
        
        #Fit local model to local data
        dense_histroy, dense_user_results = mh.compile_fit_evaluate_model(
            model=local_dense_moe_model, 
            loss=loss, 
            metrics=metrics, 
            X_train=X_train[f'user{idx+1}'],
            y_train = y_train[f'user{idx+1}'], 
            max_epochs = max_epochs, 
            batch_size=batch_size, 
            X_val=X_val[f'user{idx+1}'], 
            y_val=y_val[f'user{idx+1}'], 
            X_test=X_test[f'user{idx+1}'], 
            y_test=y_test[f'user{idx+1}'], 
            callbacks=callbacks, 
            user=f'user{idx+1}', 
            hyper=dense_moe_architecture,
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        )

        #add model weights to list        
        local_dense_moe_weights = local_dense_moe_model.get_weights()
        local_dense_moe_weight_list.append(local_dense_moe_weights)
      
        #clear session to free memory after each communication round
        K.clear_session()
    
    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights_dense_moe = sum_weights(local_dense_moe_weight_list)
    #update global model 
    global_dense_moe.set_weights(average_weights_dense_moe)
    #Save global models
    global_dense_moe.save(cwd + f"/models/FL/Dense/global_dense_model/{dense_moe_architecture}/FederatedRound{federated_round+1}")
    print("Saved Global models")

dense_all_results = pd.DataFrame(columns=["user", "architecture", "train_time", "avg_time_epoch", "mse", "rmse", "mape", "mae"])

#For each of the users
for idx in range(len(df_array)):
    print("User: ", idx+1)
    for round in range(3):
        #print("Round: ", round)
        global_dense_moe = tf.keras.models.load_model(cwd + f"/models/FL/Dense/global_dense_model/{dense_moe_architecture}/FederatedRound{federated_rounds}", compile=False)
        dense_moe_model = m1.build_topk_dense_moe_model(X_train[f'user{idx+1}'], batch_size, horizon, dense_units, num_experts, top_k, expert_units, m1)
        dense_moe_model.set_weights(global_dense_moe.get_weights())
        
        dense_histroy, dense_user_results = mh.compile_fit_evaluate_model(
            model=dense_moe_model, 
            loss=loss, 
            metrics=metrics, 
            X_train=X_train[f'user{idx+1}'],
            y_train = y_train[f'user{idx+1}'], 
            max_epochs = max_epochs, 
            batch_size=batch_size, 
            X_val=X_val[f'user{idx+1}'], 
            y_val=y_val[f'user{idx+1}'], 
            X_test=X_test[f'user{idx+1}'], 
            y_test=y_test[f'user{idx+1}'], 
            callbacks=callbacks, 
            user=f'user{idx+1}', 
            hyper=dense_moe_architecture,
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
        )
        # Add the 'architecture' column from dense_user_results to dense_results
        dense_all_results = pd.merge(dense_all_results, dense_user_results, how='outer')   

for idx in range(len(df_array)):
    new_row = {
        'architecture': dense_moe_architecture,
        'train_time': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["train_time"].mean(), 
        'avg_time_epoch' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["avg_time_epoch"].mean(),
        'mse': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mse"].mean(),
        'mse_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mse"].std(),
        'rmse': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["rmse"].mean(),
        'rmse_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["rmse"].std(),
        'mape': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mape"].mean(),
        'mape_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mape"].std(),
        'mae': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mae"].mean(),
        'mae_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mae"].std(),
    }
    dense_results.loc[len(dense_results)] = new_row

INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/TopK_Dense_MoE/FederatedRound0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/TopK_Dense_MoE/FederatedRound0\assets


Started Federated training round ---------- 1 / 5
User:  1
User:  2
User:  3
User:  4
User:  5
User:  6
User:  7
User:  8
User:  9
User:  10
User:  11
User:  12
User:  13
User:  14
User:  15
User:  16
User:  17
User:  18
User:  19
User:  20
User:  21
User:  22
User:  23
User:  24
User:  25
User:  26
User:  27
User:  28
User:  29
User:  30


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/TopK_Dense_MoE/FederatedRound1\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/TopK_Dense_MoE/FederatedRound1\assets


...

INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/TopK_Dense_MoE/FederatedRound4\assets


Saved Global models
Started Federated training round ---------- 5 / 5
User:  1
User:  2
User:  3
User:  4
User:  5
User:  6
User:  7
User:  8
User:  9
User:  10
User:  11
User:  12
User:  13
User:  14
User:  15
User:  16
User:  17
User:  18
User:  19
User:  20
User:  21
User:  22
User:  23
User:  24
User:  25
User:  26
User:  27
User:  28
User:  29
User:  30


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/TopK_Dense_MoE/FederatedRound5\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/TopK_Dense_MoE/FederatedRound5\assets


Saved Global models
User:  1
User:  2
User:  3
User:  4
User:  5
User:  6
User:  7
User:  8
User:  9
User:  10
User:  11
User:  12
User:  13
User:  14
User:  15
User:  16
User:  17
User:  18
User:  19
User:  20
User:  21
User:  22
User:  23
User:  24
User:  25
User:  26
User:  27
User:  28
User:  29
User:  30


In [7]:
dense_results.to_csv(f'evaluations/federated_learning/{dense_moe_architecture}.csv')
dense_results

,architecture,train_time,avg_time_epoch,mse,mse_std,rmse,rmse_std,mape,mape_std,mae,mae_std
0,Dense_MoE_e4d8_d16,9.268465,0.511754,0.032702,0.000444,0.180834,0.001227,1.250649e+05,12389.254496,0.128126,0.001071
1,Dense_MoE_e4d8_d16,8.605041,0.532423,0.018393,0.000401,0.135615,0.001475,1.654385e+05,40602.250888,0.081686,0.004076
2,Dense_MoE_e4d8_d16,13.408503,0.509677,0.020054,0.000587,0.141602,0.002074,4.704948e+04,2050.326539,0.078684,0.003227
3,Dense_MoE_e4d8_d16,6.529357,0.536536,0.021754,0.000949,0.147468,0.003220,1.313221e+05,11123.619260,0.063685,0.001449
4,Dense_MoE_e4d8_d16,6.800499,0.545564,0.014152,0.000337,0.118955,0.001415,1.051527e+05,26613.256395,0.076650,0.003925
5,Dense_MoE_e4d8_d16,7.582706,0.523895,0.019952,0.001053,0.141219,0.003700,7.669271e+04,16555.423893,0.093478,0.000894
6,Dense_MoE_e4d8_d16,12.690824,0.497024,0.022740,0.000921,0.150778,0.003038,9.930006e+04,8539.451449,0.085347,0.000249
7,Dense_MoE_e4d8_d16,8.062499,0.524665,0.026485,0.000438,0.162740,0.001341,2.144867e+04,19135.500169,0.106464,0.001378
8,Dense_MoE_e4d8_d16,6.512782,0.535089,0.024701,0.000680,0.157155,0.002156,9.044453e+01,10.354164,0.096917,0.004208
9,Dense_MoE_e4d8_d16,8.307605,0.525713,0.016535,0.001275,0.128526,0.004909,1.289280e+06,270636.507982,0.069106,0.002358


# Federated Learning Bidirectional LSTM Moe

In [8]:
#MOE MODEL 1 ------------------------------------------------------------------
bilstm_moe_architecture = "Soft_BiLSTM_MoE"
lstm_units = 4
num_experts = 4
expert_units = 8

#Build and save global model
global_dense_moe = m1.build_soft_biLSTM_moe_model(X_train[f'user{1}'], batch_size, horizon, lstm_units, num_experts, expert_units, m1)
global_dense_moe.save(cwd + f"/models/FL/Dense/global_dense_model/{bilstm_moe_architecture}/FederatedRound{0}")

federated_rounds = 5
for federated_round  in range(federated_rounds):
    print("Started Federated training round ----------", federated_round+1, f"/ {federated_rounds}")

    #Get global models weights
    global_dense_moe = keras.models.load_model(cwd + f"/models/FL/Dense/global_dense_model/{bilstm_moe_architecture}/FederatedRound{federated_round}", compile=False)
    global_dense_moe_weights = global_dense_moe.get_weights()

    #initial list for local model weights
    local_dense_moe_weight_list = list()

    for idx, user in enumerate(df_array): 
        print("User: ", idx+1)
        
        #build and compile local model X_train, batch_size, horizon, dense_units,  expert_units, num_experts, m1
        local_dense_moe_model = m1.build_soft_biLSTM_moe_model(X_train[f'user{idx+1}'], batch_size, horizon, lstm_units, num_experts, expert_units, m1)
        local_dense_moe_model.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=metrics)
        
        #set local model weight to the weight of the global model
        local_dense_moe_model.set_weights(global_dense_moe_weights)
        
        #Fit local model to local data
        dense_histroy, dense_user_results = mh.compile_fit_evaluate_model(
            model=local_dense_moe_model, 
            loss=loss, 
            metrics=metrics, 
            X_train=X_train[f'user{idx+1}'],
            y_train = y_train[f'user{idx+1}'], 
            max_epochs = max_epochs, 
            batch_size=batch_size, 
            X_val=X_val[f'user{idx+1}'], 
            y_val=y_val[f'user{idx+1}'], 
            X_test=X_test[f'user{idx+1}'], 
            y_test=y_test[f'user{idx+1}'], 
            callbacks=callbacks, 
            user=f'user{idx+1}', 
            hyper=bilstm_moe_architecture,
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        )

        #add model weights to list        
        local_dense_moe_weights = local_dense_moe_model.get_weights()
        local_dense_moe_weight_list.append(local_dense_moe_weights)
      
        #clear session to free memory after each communication round
        K.clear_session()
    
    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights_dense_moe = sum_weights(local_dense_moe_weight_list)
    #update global model 
    global_dense_moe.set_weights(average_weights_dense_moe)
    #Save global models
    global_dense_moe.save(cwd + f"/models/FL/Dense/global_dense_model/{bilstm_moe_architecture}/FederatedRound{federated_round+1}")
    print("Saved Global models")

dense_all_results = pd.DataFrame(columns=["user", "architecture", "train_time", "avg_time_epoch", "mse", "rmse", "mape", "mae"])

#For each of the users
for idx in range(len(df_array)):
    print("User: ", idx+1)
    for round in range(3):
        #print("Round: ", round)
        global_dense_moe = tf.keras.models.load_model(cwd + f"/models/FL/Dense/global_dense_model/{bilstm_moe_architecture}/FederatedRound{federated_rounds}", compile=False)
        dense_moe_model = m1.build_soft_biLSTM_moe_model(X_train[f'user{idx+1}'], batch_size, horizon, lstm_units, num_experts, expert_units, m1)
        dense_moe_model.set_weights(global_dense_moe.get_weights())
        
        dense_histroy, dense_user_results = mh.compile_fit_evaluate_model(
            model=dense_moe_model, 
            loss=loss, 
            metrics=metrics, 
            X_train=X_train[f'user{idx+1}'],
            y_train = y_train[f'user{idx+1}'], 
            max_epochs = max_epochs, 
            batch_size=batch_size, 
            X_val=X_val[f'user{idx+1}'], 
            y_val=y_val[f'user{idx+1}'], 
            X_test=X_test[f'user{idx+1}'], 
            y_test=y_test[f'user{idx+1}'], 
            callbacks=callbacks, 
            user=f'user{idx+1}', 
            hyper=bilstm_moe_architecture,
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
        )
        # Add the 'architecture' column from dense_user_results to dense_results
        dense_all_results = pd.merge(dense_all_results, dense_user_results, how='outer')   

for idx in range(len(df_array)):
    new_row = {
        'architecture': bilstm_moe_architecture,
        'train_time': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["train_time"].mean(), 
        'avg_time_epoch' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["avg_time_epoch"].mean(),
        'mse': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mse"].mean(),
        'mse_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mse"].std(),
        'rmse': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["rmse"].mean(),
        'rmse_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["rmse"].std(),
        'mape': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mape"].mean(),
        'mape_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mape"].std(),
        'mae': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mae"].mean(),
        'mae_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mae"].std(),
    }
    dense_results.loc[len(dense_results)] = new_row

INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/Soft_BiLSTM_MoE/FederatedRound0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/Soft_BiLSTM_MoE/FederatedRound0\assets


Started Federated training round ---------- 1 / 5
User:  1
User:  2
User:  3
User:  4
User:  5
User:  6
User:  7
User:  8
User:  9
User:  10
User:  11
User:  12
User:  13
User:  14
User:  15
User:  16
User:  17
User:  18
User:  19
User:  20
User:  21
User:  22
User:  23
User:  24
User:  25
User:  26
User:  27
User:  28
User:  29
User:  30


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/Soft_BiLSTM_MoE/FederatedRound1\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/Soft_BiLSTM_MoE/FederatedRound1\assets


...

INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/Soft_BiLSTM_MoE/FederatedRound4\assets


Saved Global models
Started Federated training round ---------- 5 / 5
User:  1
User:  2
User:  3
User:  4
User:  5
User:  6
User:  7
User:  8
User:  9
User:  10
User:  11
User:  12
User:  13
User:  14
User:  15
User:  16
User:  17
User:  18
User:  19
User:  20
User:  21
User:  22
User:  23
User:  24
User:  25
User:  26
User:  27
User:  28
User:  29
User:  30


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/Soft_BiLSTM_MoE/FederatedRound5\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/Soft_BiLSTM_MoE/FederatedRound5\assets


Saved Global models
User:  1
User:  2
User:  3
User:  4
User:  5
User:  6
User:  7
User:  8
User:  9
User:  10
User:  11
User:  12
User:  13
User:  14
User:  15
User:  16
User:  17
User:  18
User:  19
User:  20
User:  21
User:  22
User:  23
User:  24
User:  25
User:  26
User:  27
User:  28
User:  29
User:  30


In [9]:
dense_results.to_csv(f'evaluations/federated_learning/{bilstm_moe_architecture}.csv')
dense_results

,architecture,train_time,avg_time_epoch,mse,mse_std,rmse,rmse_std,mape,mape_std,mae,mae_std
0,Dense_MoE_e4d8_d16,9.268465,0.511754,0.032702,0.000444,0.180834,0.001227,1.250649e+05,12389.254496,0.128126,0.001071
1,Dense_MoE_e4d8_d16,8.605041,0.532423,0.018393,0.000401,0.135615,0.001475,1.654385e+05,40602.250888,0.081686,0.004076
2,Dense_MoE_e4d8_d16,13.408503,0.509677,0.020054,0.000587,0.141602,0.002074,4.704948e+04,2050.326539,0.078684,0.003227
3,Dense_MoE_e4d8_d16,6.529357,0.536536,0.021754,0.000949,0.147468,0.003220,1.313221e+05,11123.619260,0.063685,0.001449
4,Dense_MoE_e4d8_d16,6.800499,0.545564,0.014152,0.000337,0.118955,0.001415,1.051527e+05,26613.256395,0.076650,0.003925
...,...,...,...,...,...,...,...,...,...,...,...
85,Soft_BiLSTM_MoE,28.224068,2.223882,0.023819,0.000531,0.154327,0.001721,8.562537e+04,12560.438081,0.091369,0.002108
86,Soft_BiLSTM_MoE,24.598141,2.220676,0.017793,0.000358,0.133385,0.001346,2.007702e+05,23816.305698,0.090325,0.000598
87,Soft_BiLSTM_MoE,31.365006,2.127270,0.010638,0.000242,0.103134,0.001175,4.495762e+04,2103.294051,0.044651,0.001629
88,Soft_BiLSTM_MoE,50.089800,2.144714,0.016531,0.000184,0.128571,0.000718,5.573850e+05,47902.447782,0.093014,0.000766


In [10]:
#MOE MODEL 1 ------------------------------------------------------------------
bilstm_moe_architecture = "TopK_BiLSTM_MoE"
lstm_units = 8

num_experts = 4
top_k = 2
expert_units = 8

#Build and save global model
global_dense_moe = m1.build_topk_bilstm_moe_model(X_train[f'user{1}'], batch_size, horizon, lstm_units, num_experts, top_k, expert_units, m1)
global_dense_moe.save(cwd + f"/models/FL/Dense/global_dense_model/{bilstm_moe_architecture}/FederatedRound{0}")

federated_rounds = 5
for federated_round  in range(federated_rounds):
    print("Started Federated training round ----------", federated_round+1, f"/ {federated_rounds}")

    #Get global models weights
    global_dense_moe = keras.models.load_model(cwd + f"/models/FL/Dense/global_dense_model/{bilstm_moe_architecture}/FederatedRound{federated_round}", compile=False)
    global_dense_moe_weights = global_dense_moe.get_weights()

    #initial list for local model weights
    local_dense_moe_weight_list = list()

    for idx, user in enumerate(df_array): 
        print("User: ", idx+1)
        
        #build and compile local model X_train, batch_size, horizon, dense_units,  expert_units, num_experts, m1
        local_dense_moe_model = m1.build_topk_bilstm_moe_model(X_train[f'user{idx+1}'], batch_size, horizon, lstm_units, num_experts, top_k, expert_units, m1)
        local_dense_moe_model.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=metrics)
        
        #set local model weight to the weight of the global model
        local_dense_moe_model.set_weights(global_dense_moe_weights)
        
        #Fit local model to local data
        dense_histroy, dense_user_results = mh.compile_fit_evaluate_model(
            model=local_dense_moe_model, 
            loss=loss, 
            metrics=metrics, 
            X_train=X_train[f'user{idx+1}'],
            y_train = y_train[f'user{idx+1}'], 
            max_epochs = max_epochs, 
            batch_size=batch_size, 
            X_val=X_val[f'user{idx+1}'], 
            y_val=y_val[f'user{idx+1}'], 
            X_test=X_test[f'user{idx+1}'], 
            y_test=y_test[f'user{idx+1}'], 
            callbacks=callbacks, 
            user=f'user{idx+1}', 
            hyper=bilstm_moe_architecture,
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        )

        #add model weights to list        
        local_dense_moe_weights = local_dense_moe_model.get_weights()
        local_dense_moe_weight_list.append(local_dense_moe_weights)
      
        #clear session to free memory after each communication round
        K.clear_session()
    
    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights_dense_moe = sum_weights(local_dense_moe_weight_list)
    #update global model 
    global_dense_moe.set_weights(average_weights_dense_moe)
    #Save global models
    global_dense_moe.save(cwd + f"/models/FL/Dense/global_dense_model/{bilstm_moe_architecture}/FederatedRound{federated_round+1}")
    print("Saved Global models")

dense_all_results = pd.DataFrame(columns=["user", "architecture", "train_time", "avg_time_epoch", "mse", "rmse", "mape", "mae"])

#For each of the users
for idx in range(len(df_array)):
    print("User: ", idx+1)
    for round in range(3):
        #print("Round: ", round)
        global_dense_moe = tf.keras.models.load_model(cwd + f"/models/FL/Dense/global_dense_model/{bilstm_moe_architecture}/FederatedRound{federated_rounds}", compile=False)
        dense_moe_model = m1.build_topk_bilstm_moe_model(X_train[f'user{idx+1}'], batch_size, horizon, lstm_units, num_experts, top_k, expert_units, m1)
        dense_moe_model.set_weights(global_dense_moe.get_weights())
        
        dense_histroy, dense_user_results = mh.compile_fit_evaluate_model(
            model=dense_moe_model, 
            loss=loss, 
            metrics=metrics, 
            X_train=X_train[f'user{idx+1}'],
            y_train = y_train[f'user{idx+1}'], 
            max_epochs = max_epochs, 
            batch_size=batch_size, 
            X_val=X_val[f'user{idx+1}'], 
            y_val=y_val[f'user{idx+1}'], 
            X_test=X_test[f'user{idx+1}'], 
            y_test=y_test[f'user{idx+1}'], 
            callbacks=callbacks, 
            user=f'user{idx+1}', 
            hyper=bilstm_moe_architecture,
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
        )
        # Add the 'architecture' column from dense_user_results to dense_results
        dense_all_results = pd.merge(dense_all_results, dense_user_results, how='outer')   

for idx in range(len(df_array)):
    new_row = {
        'architecture': bilstm_moe_architecture,
        'train_time': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["train_time"].mean(), 
        'avg_time_epoch' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["avg_time_epoch"].mean(),
        'mse': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mse"].mean(),
        'mse_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mse"].std(),
        'rmse': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["rmse"].mean(),
        'rmse_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["rmse"].std(),
        'mape': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mape"].mean(),
        'mape_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mape"].std(),
        'mae': dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mae"].mean(),
        'mae_std' : dense_all_results[dense_all_results["user"]==f"user{idx+1}"]["mae"].std(),
    }
    dense_results.loc[len(dense_results)] = new_row

INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/TopK_BiLSTM_MoE/FederatedRound0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/TopK_BiLSTM_MoE/FederatedRound0\assets


Started Federated training round ---------- 1 / 5
User:  1
User:  2
User:  3
User:  4
User:  5
User:  6
User:  7
User:  8
User:  9
User:  10
User:  11
User:  12
User:  13
User:  14
User:  15
User:  16
User:  17
User:  18
User:  19
User:  20
User:  21
User:  22
User:  23
User:  24
User:  25
User:  26
User:  27
User:  28
User:  29
User:  30


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/TopK_BiLSTM_MoE/FederatedRound1\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/TopK_BiLSTM_MoE/FederatedRound1\assets


...

INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/TopK_BiLSTM_MoE/FederatedRound4\assets


Saved Global models
Started Federated training round ---------- 5 / 5
User:  1
User:  2
User:  3
User:  4
User:  5
User:  6
User:  7
User:  8
User:  9
User:  10
User:  11
User:  12
User:  13
User:  14
User:  15
User:  16
User:  17
User:  18
User:  19
User:  20
User:  21
User:  22
User:  23
User:  24
User:  25
User:  26
User:  27
User:  28
User:  29
User:  30


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/TopK_BiLSTM_MoE/FederatedRound5\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/TopK_BiLSTM_MoE/FederatedRound5\assets


Saved Global models
User:  1
User:  2
User:  3
User:  4
User:  5
User:  6
User:  7
User:  8
User:  9
User:  10
User:  11
User:  12
User:  13
User:  14
User:  15
User:  16
User:  17
User:  18
User:  19
User:  20
User:  21
User:  22
User:  23
User:  24
User:  25
User:  26
User:  27
User:  28
User:  29
User:  30


In [11]:
dense_results.to_csv(f'evaluations/federated_learning/{bilstm_moe_architecture}.csv')
dense_results

,architecture,train_time,avg_time_epoch,mse,mse_std,rmse,rmse_std,mape,mape_std,mae,mae_std
0,Dense_MoE_e4d8_d16,9.268465,0.511754,0.032702,0.000444,0.180834,0.001227,1.250649e+05,12389.254496,0.128126,0.001071
1,Dense_MoE_e4d8_d16,8.605041,0.532423,0.018393,0.000401,0.135615,0.001475,1.654385e+05,40602.250888,0.081686,0.004076
2,Dense_MoE_e4d8_d16,13.408503,0.509677,0.020054,0.000587,0.141602,0.002074,4.704948e+04,2050.326539,0.078684,0.003227
3,Dense_MoE_e4d8_d16,6.529357,0.536536,0.021754,0.000949,0.147468,0.003220,1.313221e+05,11123.619260,0.063685,0.001449
4,Dense_MoE_e4d8_d16,6.800499,0.545564,0.014152,0.000337,0.118955,0.001415,1.051527e+05,26613.256395,0.076650,0.003925
...,...,...,...,...,...,...,...,...,...,...,...
115,TopK_BiLSTM_MoE,34.465111,2.508670,0.024167,0.000862,0.155441,0.002777,9.147803e+04,15058.487143,0.095867,0.002389
116,TopK_BiLSTM_MoE,34.392478,2.565804,0.020128,0.000823,0.141853,0.002916,1.502659e+05,13199.168307,0.096257,0.001110
117,TopK_BiLSTM_MoE,34.943494,2.481733,0.010847,0.000268,0.104142,0.001284,5.056263e+04,713.832238,0.045670,0.000489
118,TopK_BiLSTM_MoE,50.363433,2.457005,0.017376,0.000487,0.131809,0.001850,3.887447e+05,153010.933215,0.090892,0.002538


In [12]:
print("Soft MoE Dense: ", dense_results[0:29]["mse"].mean())
print("TopK MoE Dense: ", dense_results[30:59]["mse"].mean())
print("Soft MoE Dense: ", dense_results[60:89]["mse"].mean())
print("Soft MoE Dense: ", dense_results[90:119]["mse"].mean())

Soft MoE Dense:  0.01846480392048071
TopK MoE Dense:  0.019183966357946055
Soft MoE Dense:  0.01744719374196968
Soft MoE Dense:  0.018114131013595165
